# VacationPy

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,11.38,80,92,3.98,NZ,1681359721
1,1,rikitea,-23.1203,-134.9692,25.42,69,83,5.06,PF,1681359721
2,2,east london,-33.0153,27.9116,15.52,82,0,3.09,ZA,1681359721
3,3,vicuna,-30.0319,-70.7081,9.85,62,77,0.33,CL,1681359721
4,4,ushuaia,-54.8000,-68.3000,4.81,75,75,7.20,AR,1681359722


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha= 0.5
)

#play the map
map_city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
#A max temperature lower than 27 degrees but higher than 21, windspeed < 4.5 m/s, zero cloudiness
ideal_conditions_cities  = city_data_df.loc[(city_data_df['Max Temp'] < 28) & (city_data_df['Max Temp'] >= 21) & (city_data_df['Wind Speed'] < 4.5)& (city_data_df['Cloudiness'] == 0),:]

# Drop any rows with null values
ideal_condtions_citites = ideal_conditions_cities.dropna(how='any')

# Display sample data
ideal_conditions_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,jamestown,42.0970,-79.2353,21.77,61,0,3.60,US,1681359432
21,21,hasaki,35.7333,140.8333,22.34,35,0,4.18,JP,1681359729
35,35,boa vista,2.8197,-60.6733,27.99,69,0,1.54,BR,1681359567
133,133,saint-pierre,-21.3393,55.4781,27.54,65,0,1.54,RE,1681359767
201,201,puerto escondido,15.8500,-97.0667,26.46,73,0,4.12,MX,1681359792
249,249,mayo,38.8876,-76.5119,21.39,52,0,2.06,US,1681359811
267,267,celestun,20.8667,-90.4000,23.57,69,0,2.48,MX,1681359817
314,314,progreso,21.2833,-89.6667,23.39,69,0,1.89,MX,1681359550
332,332,acapulco,16.8634,-99.8901,27.90,69,0,1.03,MX,1681359627
359,359,melville,40.7934,-73.4151,23.19,56,0,0.00,US,1681359598


In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_conditions_cities.loc[:,('City','Country','Lat','Lng','Humidity')]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
19,jamestown,US,42.0970,-79.2353,61,
21,hasaki,JP,35.7333,140.8333,35,
35,boa vista,BR,2.8197,-60.6733,69,
133,saint-pierre,RE,-21.3393,55.4781,65,
201,puerto escondido,MX,15.8500,-97.0667,73,
249,mayo,US,38.8876,-76.5119,52,
267,celestun,MX,20.8667,-90.4000,69,
314,progreso,MX,21.2833,-89.6667,69,
332,acapulco,MX,16.8634,-99.8901,69,
359,melville,US,40.7934,-73.4151,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
categories= "accommodation.hotel"
limit=20

params = {
    "categories": categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude=row[3]
    latitude=row[2]
                
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    print(longitude,latitude)
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
-79.2353 42.097
jamestown - nearest hotel: DoubleTree Jamestown
140.8333 35.7333
hasaki - nearest hotel: 大新旅館
-60.6733 2.8197
boa vista - nearest hotel: Aipana Plaza Hotel
55.4781 -21.3393
saint-pierre - nearest hotel: Tropic Hotel
-97.0667 15.85
puerto escondido - nearest hotel: Hotel Barlovento
-76.5119 38.8876
mayo - nearest hotel: The Westin
-90.4 20.8667
celestun - nearest hotel: Hotel Flamingo Playa
-89.6667 21.2833
progreso - nearest hotel: Centro Vacacional Obrero CTM
-99.8901 16.8634
acapulco - nearest hotel: Hotel del Valle
-73.4151 40.7934
melville - nearest hotel: Marriott Melville Long Island


,City,Country,Lat,Lng,Humidity,Hotel Name
19,jamestown,US,42.0970,-79.2353,61,DoubleTree Jamestown
21,hasaki,JP,35.7333,140.8333,35,大新旅館
35,boa vista,BR,2.8197,-60.6733,69,Aipana Plaza Hotel
133,saint-pierre,RE,-21.3393,55.4781,65,Tropic Hotel
201,puerto escondido,MX,15.8500,-97.0667,73,Hotel Barlovento
249,mayo,US,38.8876,-76.5119,52,The Westin
267,celestun,MX,20.8667,-90.4000,69,Hotel Flamingo Playa
314,progreso,MX,21.2833,-89.6667,69,Centro Vacacional Obrero CTM
332,acapulco,MX,16.8634,-99.8901,69,Hotel del Valle
359,melville,US,40.7934,-73.4151,56,Marriott Melville Long Island


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_hotel_plot = hotel_df.hvplot.points(
    "Lng",
     "Lat",
     geo = True,
     tiles = "OSM",
     frame_width = 800,
     frame_height = 600,
     size="Humidity",
     scale = 1,
     color = "City",
     alpha= 0.5,
     hover_cols=['Country','Hotel Name']
 )

# Display the map
map_hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)